In [22]:
import pandas as pd
import re
df = pd.read_json("/content/ExtractedData.json")

**Data Cleaning and Preprocessing**


*Data Preview*

In [23]:
print(df.head())
print(df.tail())

                                            disorder  \
0  الاكتئاب واضطراب اختلال المزاج عند الأطفال وال...   
1                 اضطراب القلق المُعمَّم عند الأطفال   
2                               الاضطراب ثنائي القطب   
3  إيذاء النَّفس غير الانتحاريّ عندَ الأطفال والم...   
4           رُهَاب المَيادين عند الأطفال والمُراهقين   

                                          definition  \
0  [ينطوي الاكتئاب على شعور بالحزن (أو الهياج عند...   
1  [ينطوي اضطراب القلق المعمم على عصبية شديدة ومس...   
2  [الاضطراب ثنائي القطب هو اضطراب نفسي يعاني فيه...   
3  [يشير إيذاء النَّفس غير الانتحاريّ إلى الأذى ا...   
4  [رُهَاب الميادين هو خوف مستمر من التواجد في أم...   

                                             symptom  \
0  [الشعور بالحزن أو الهيجان ،, عدم الاهتمام بالن...   
1  [القلق المستمر بشأن العديد من الأشياء والنشاطا...   
2  [أعراض الاكتئاب ،: الشعور بالحزن الشديد، قلة ا...   
3  [التسبب بالضرر للذات في كثيرٍ من الأحيان ،, اس...   
4  [خوف استخدام وسائل النقل العام ،, خوف التوا

In [24]:
print(df.shape)

(86, 8)


In [25]:
print(df.columns)

Index(['disorder', 'definition', 'symptom', 'diagnosis', 'treatment', 'cause',
       'Disorder', 'disorder_url'],
      dtype='object')


*Remove duplicate column*

In [26]:
df['disorder'] = df['disorder'].combine_first(df['Disorder'])

# Drop the 'Disorder' column
df.drop(columns=['Disorder'], inplace=True)

print(df.columns)

Index(['disorder', 'definition', 'symptom', 'diagnosis', 'treatment', 'cause',
       'disorder_url'],
      dtype='object')


*Check missing values*

In [27]:
print(df.isnull().sum())

disorder         0
definition       0
symptom          0
diagnosis       24
treatment        1
cause           52
disorder_url    83
dtype: int64


 *Imputation with placeholder text*

In [28]:
df['diagnosis'].fillna('لم يتم عرض نتائج تشخيصية لحد الان', inplace=True)
df['cause'].fillna('لم يتم توفير معلومات حول اسباب هذا الاضطراب او المرض', inplace=True)
df['treatment'].fillna('لم يتم ايجاد علاج محدد لحد الان', inplace=True)

print(df.isnull().sum())

disorder         0
definition       0
symptom          0
diagnosis        0
treatment        0
cause            0
disorder_url    83
dtype: int64


<ipython-input-28-3dac2398afa3>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['diagnosis'].fillna('لم يتم عرض نتائج تشخيصية لحد الان', inplace=True)
<ipython-input-28-3dac2398afa3>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value,

*Drop irrelevant columns*

In [29]:
df.drop(columns=['disorder_url','definition', 'diagnosis', 'cause'], inplace=True)
print(df.columns)

Index(['disorder', 'symptom', 'treatment'], dtype='object')


*Check duplicated content*

In [30]:
duplicates = df[df.duplicated(subset=['disorder'], keep=False)]
print(duplicates)


                                        disorder  \
32                                شَهوة الغَرائب   
57                                شَهوة الغَرائب   
81  اضطراب السُّلوك التكراريّ المركِّز على الجسم   
82  اضطراب السُّلوك التكراريّ المركِّز على الجسم   

                                              symptom  \
32  [ويؤدي هذا الاضطرابُ إلى تناول الشخص المصاب به...   
57  [يؤدي هذا الاضطراب إلى تناول الشخص مواد غير صا...   
81  [وبعضُ الأشخاص يفعلون هذه الأنشطة تلقائيًا إلى...   
82  [وبعضُ الأشخاص يفعلون هذه الأنشطة تلقائيًا إلى...   

                                            treatment  
32  [التعديل السلوكي للشخص المصاب . ،, مُعالَجة ال...  
57  [التعديل السلوكي للشخص المصاب (قد يكون فعالاً ...  
81             [الأدوية،, العلاج السُّلُوكي المعرفي،]  
82             [الأدوية،, العلاج السُّلُوكي المعرفي،]  


In [31]:
df.drop_duplicates(subset=['disorder'], keep='first', inplace=True)

print(df[df['disorder'].isin(duplicates['disorder'].unique())])

                                        disorder  \
32                                شَهوة الغَرائب   
81  اضطراب السُّلوك التكراريّ المركِّز على الجسم   

                                              symptom  \
32  [ويؤدي هذا الاضطرابُ إلى تناول الشخص المصاب به...   
81  [وبعضُ الأشخاص يفعلون هذه الأنشطة تلقائيًا إلى...   

                                            treatment  
32  [التعديل السلوكي للشخص المصاب . ،, مُعالَجة ال...  
81             [الأدوية،, العلاج السُّلُوكي المعرفي،]  


*Remove Non-arabic Words*

In [32]:
def remove_non_arabic(text):

    text = str(text)
    arabic_pattern = re.compile(
        r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF\s]'
    )

    return arabic_pattern.sub('', text)

df = df.apply(lambda col: col.map(remove_non_arabic))
print(df.head)

<bound method NDFrame.head of                                              disorder  \
0   الاكتئاب واضطراب اختلال المزاج عند الأطفال وال...   
1                  اضطراب القلق المُعمَّم عند الأطفال   
2                                الاضطراب ثنائي القطب   
3   إيذاء النَّفس غير الانتحاريّ عندَ الأطفال والم...   
4            رُهَاب المَيادين عند الأطفال والمُراهقين   
..                                                ...   
80                                   اضطرابُ الاكتناز   
81       اضطراب السُّلوك التكراريّ المركِّز على الجسم   
83                  اضطراب شدِّ الشعر هَوَسُ النَّتْف   
84                            اضطرابُ خدش تقشير الجلد   
85                       الاضطرابُ الوسواسي القهري      

                                              symptom  \
0   الشعور بالحزن أو الهيجان ، عدم الاهتمام بالنشا...   
1   القلق المستمر بشأن العديد من الأشياء والنشاطات...   
2   أعراض الاكتئاب ، الشعور بالحزن الشديد، قلة الا...   
3   التسبب بالضرر للذات في كثيرٍ من الأحيان ، استخ...   


*Remove Punctuation*

In [33]:
def remove_punctuation_selective(text):
    text = str(text)

    punctuation_to_keep = r'",/،.؟'

    text = re.sub(fr'[^\u0600-\u06FF\s{punctuation_to_keep}]', '', text)

    text = re.sub(r'\s+', ' ', text).strip()
    return text

df = df.apply(lambda col: col.map(remove_punctuation_selective))
print(df.head())

                                            disorder  \
0  الاكتئاب واضطراب اختلال المزاج عند الأطفال وال...   
1                 اضطراب القلق المُعمَّم عند الأطفال   
2                               الاضطراب ثنائي القطب   
3  إيذاء النَّفس غير الانتحاريّ عندَ الأطفال والم...   
4           رُهَاب المَيادين عند الأطفال والمُراهقين   

                                             symptom  \
0  الشعور بالحزن أو الهيجان ، عدم الاهتمام بالنشا...   
1  القلق المستمر بشأن العديد من الأشياء والنشاطات...   
2  أعراض الاكتئاب ، الشعور بالحزن الشديد، قلة الا...   
3  التسبب بالضرر للذات في كثيرٍ من الأحيان ، استخ...   
4  خوف استخدام وسائل النقل العام ، خوف التواجد في...   

                                           treatment  
0  العلاج النفسي ،ومضادات الاكتئاب بالنسبة إلى مع...  
1   التدريب على الاسترخاء ، الأدوية في بعض الأحيان ،  
2  وصف الأدوية مثل الليثيوم أو فالبروات ، المشورة...  
3  العلاج الفردي وأحيانًا الجماعي ، تعليم إدراك ا...  
4  العلاج السُّلُوكي كنهج فعَّال لتخفيف الأعراض ،..

*Arabic Normalization*

In [34]:
def normalize_arabic(text):
    if isinstance(text, str):
        text = re.sub(r"[أإآ]", "ا", text)  # Convert أ, إ, آ to ا
        text = re.sub(r"ى", "ي", text)      # Convert ى to ي
        text = re.sub(r"[ؤ]", "و", text)    # Normalize Hamza on Waw to Waw
        text = re.sub(r"ـ", "", text)       # Remove Tatweel (ـ)
        return text.strip()
    return text

df = df.apply(lambda col: col.map(normalize_arabic))
print(df.head())


                                            disorder  \
0  الاكتئاب واضطراب اختلال المزاج عند الاطفال وال...   
1                 اضطراب القلق المُعمَّم عند الاطفال   
2                               الاضطراب ثنائي القطب   
3  ايذاء النَّفس غير الانتحاريّ عندَ الاطفال والم...   
4           رُهَاب المَيادين عند الاطفال والمُراهقين   

                                             symptom  \
0  الشعور بالحزن او الهيجان ، عدم الاهتمام بالنشا...   
1  القلق المستمر بشان العديد من الاشياء والنشاطات...   
2  اعراض الاكتئاب ، الشعور بالحزن الشديد، قلة الا...   
3  التسبب بالضرر للذات في كثيرٍ من الاحيان ، استخ...   
4  خوف استخدام وسائل النقل العام ، خوف التواجد في...   

                                           treatment  
0  العلاج النفسي ،ومضادات الاكتئاب بالنسبة الي مع...  
1   التدريب علي الاسترخاء ، الادوية في بعض الاحيان ،  
2  وصف الادوية مثل الليثيوم او فالبروات ، المشورة...  
3  العلاج الفردي واحيانًا الجماعي ، تعليم ادراك ا...  
4  العلاج السُّلُوكي كنهج فعَّال لتخفيف الاعراض ،..

*Remove diacritics (Tashkeel)*

In [35]:
def remove_tashkeel(text):
    if isinstance(text, str):
        return re.sub(r"[\u064B-\u065F\u0610-\u061A\u0670]", "", text)
    return text

df = df.apply(lambda col: col.map(remove_tashkeel))

print(df.head())


                                            disorder  \
0  الاكتئاب واضطراب اختلال المزاج عند الاطفال وال...   
1                    اضطراب القلق المعمم عند الاطفال   
2                               الاضطراب ثنائي القطب   
3   ايذاء النفس غير الانتحاري عند الاطفال والمراهقين   
4               رهاب الميادين عند الاطفال والمراهقين   

                                             symptom  \
0  الشعور بالحزن او الهيجان ، عدم الاهتمام بالنشا...   
1  القلق المستمر بشان العديد من الاشياء والنشاطات...   
2  اعراض الاكتئاب ، الشعور بالحزن الشديد، قلة الا...   
3  التسبب بالضرر للذات في كثير من الاحيان ، استخد...   
4  خوف استخدام وسائل النقل العام ، خوف التواجد في...   

                                           treatment  
0  العلاج النفسي ،ومضادات الاكتئاب بالنسبة الي مع...  
1   التدريب علي الاسترخاء ، الادوية في بعض الاحيان ،  
2  وصف الادوية مثل الليثيوم او فالبروات ، المشورة...  
3  العلاج الفردي واحيانا الجماعي ، تعليم ادراك ال...  
4  العلاج السلوكي كنهج فعال لتخفيف الاعراض ، التد..

*export the cleaned data*


In [36]:
df.to_json("/content/CleanedData.json", orient="records", force_ascii=False, indent=4)

print("Cleaned DataFrame has been exported to '/content/CleanedData.json'")

Cleaned DataFrame has been exported to '/content/CleanedData.json'


*Text Segmentation*

In [37]:
import json
import re

def segment_text(text):
    if not text:
        return []
    sentences = re.split(r'(?<=[.!؟,،])\s+', text)
    return [s.strip() for s in sentences if s.strip()]

with open("CleanedData.json", "r", encoding="utf-8") as f:
    CleanedData = json.load(f)

segmented_data = []
for entry in CleanedData:
    segmented_entry = {
        "disorder": segment_text(entry["disorder"]),
        "symptom": segment_text(entry["symptom"]),
        "treatment": segment_text(entry["treatment"])
    }
    segmented_data.append(segmented_entry)

with open("segmented_data.json", "w", encoding="utf-8") as f:
    json.dump(segmented_data, f, ensure_ascii=False, indent=4)

print("Segmentation complete! Results saved to 'segmented_data.json'.")

Segmentation complete! Results saved to 'segmented_data.json'.


*Define NE (Named Entities)*

In [17]:
import json

named_entities = {
    "disorder": [
      "اضطراب اختلال المزاج", "الاكتئاب", "اضطراب القلق المعمم", "الاضطراب ثنائي القطب", "ايذاء النفس غير الانتحاري", "رهاب الميادين", "اضطراب التصرف", "اضطراب المعارض المتحدي", "السلوك الانتحاري", "اضطراب قلق الانفصال", "تجنب المدرسة", "اضطراب نقص الانتباه وفرط النشاط", "اضطراب طيف التوحد", "عسر القراءة", "الاعاقة الذهنية", "نوبات الغضب", "الكرب", "اضطراب ما بعد الصدمة", "اضطراب تبدد الشخصية و تبدد المحيط", "اضطراب الشخصية الحدي", "اضطرابات التاقلم", "اضطراب الشخصية النرجسية", "اضطراب الشخصية المعادية للمجتمع", "اضطراب الهوية التفارقي", "الاضطراب الاجتراري", "اضطراب الشخصية التجنبي", "اضطراب الشخصية الاتكالية", "اضطراب الشخصية الزوراني", "اضطراب الشخصية التمثيلي", "اضطراب الشخصية الفصامي النمط", "الفصام", "اضطراب الهلع", "اضطراب الاعراض الجسدية", "اضطراب تجنب و تقييد تناول الطعام", "شهوة الغرائب", "اضطراب الاحالة الشمية", "فواصل انحباس التنفس", "اضطراب التلصص او البصبصة", "اضطرابات القلق", "متلازمة توريت", "اضطرابات العرة", "اضطراب الاستعراء", "اضطراب الفرك", "اضطراب العرض الجسدي", "الاضطراب المفتعل المفروض علي الغير", "اضطراب ذهاني قصير الامد", "الذهان", "نوبة مصحوبة بغيبة", "الصمت الانتقائي", "جنون العظمة", "الشخصية التبعية", "الشخصية الاضطهادية البارانوية", "السير خلال النوم", "الاضطرابات الذهانية المحرضة بالعقاقير والمخدرات", "اضطراب لبسة الجنس الاخر", "اضطراب الشخصية الانهزامية", "متلازمة ستوكهولم", "متلازمة جانسر", "عدم التوافق الجندري وعدم الرضا الجندري", "حالات الرهاب النوعية", "تعذر الاداء النطقي بالطفولة", "ايذاء النفس او جرح النفس", "الحبسة", "التلعثم", "الاعتلال الدماغي الرضي المزمن", "اضطرابات انشقاقية", "الاضطراب الانفجاري المتقطع", "الارق", "اضطرابات النوم", "اضطرابات التعلم", "اضطرابات الاكل", "اضطرابات التكيف", "اضطراب قلق المرض", "اضطراب عشق الاطفال", "اضطراب القلق الاجتماعي", "اضطراب الشخصية المعادية للمجتمع", "اضطراب التعلق التفاعلي", "اضطراب النهم للطعام", "النهام العصبي", "فقدان الشهية او القهام العصبي", "الاكتئاب", "اضطراب الاكتناز", "اضطراب السلوك التكراري المركز علي الجسم", "اضطراب شد الشعر هوس النتف", "اضطراب خدش تقشير الجلد", "الاضطراب الوسواسي القهري", "الكذب المرضي"
    ],
    "symptom": [
      "الحزن", "الهيجان", "عدم الاهتمام بالنشاطات المفضلة،", "الابتعاد عن الاصدقاء", "الشعور بالرفض،", "الشعور بالتعب", "عدم امتلاك اية طاقة،", "عدم النوم بشكل جيد،", "الكوابيس", "النوم لفترة طويلة جدا،", "اللوم علي الذات،", "نقص الشهية والوزن،", "التفكير في الموت او الانتحار،", "ضعف الاداء المدرسي،", "مشاكل في التفكير والتركيز", "القلق", "فرط في النشاط،", "تململ", "تهيج", "عصبية،،", "تعرق مفرط،", "الارهاق", "مشاكل في النوم،", "الم المعدة", "توتر", "الام العضلات", "الصداع", "الاكتئاب", "التفكير البطيء،", "النوم المفرط،", "قلة الاهتمام،", "الهوس", "ثقة مفرطة،", "زيادة الكلام،", "نوم اقل،", "طاقة عالية،", "ضرر للذات", "ضعف الارتباط الاجتماعي", "افكار انتحارية،", "خوف استخدام وسائل النقل العام", "خوف التواجد في الاماكن المكشوفة،", "خوف التواجد خارج المنزل،", "انانية", "يكذبون", "يتنمرون", "لا يهتمون بمشاعر الاخرين،", "يتعاطون عقاقير،", "يغشون", "يسرقون", "يهددون الاخرين", "يدخلون في شجارات،", "يخربون الممتلكات،", "عدوانيين", "يهربون من المنزل", "يتغيبون عن المدرسة،", "يزعجون الاشخاص،", "يلقون اللوم،", "الجدال", "غاضبين", "مستائين", "تقلبات شديدة في المزاج،", "صعوبة في النوم،", "علامات متدنية،", "كرب", "نحيب", "تضرع", "قلق", "الخوف من فقدان", "صعوبة في النوم", "رفض الذهاب الي المدرسة", "الصداع", "الم المعدة،", "غثيان،", "اعراض معدية", "اعراض جسدية", "صعوبة في الانتباه،", "التردد المتكرر،", "نشاط بدني مفرط،", "نسيان الاشياء،", "تشتت الانتباه،", "اندفاع", "عدم الاصغاء عند الحديث،", "صعوبة في التواصل،", "سلوكيات متكررة،", "حساسية مفرطة", "استجابة باهتة،", "تجنب التواصل البصري،", "تاخر في اكتساب اللغة،", "تاخرا في الكلام،", "صعوبة في نطق،", "مشاكل ربط الكلمات،", "صعوبة تذكر اسماء الحروف والارقام والالوان،", "التردد في اختيار الكلمات،", "الذاكرة قصيرة", "ترديد الكلمات،", "ملامح وجه غير طبيعية،", "تشوهات في اليدين او القدمين،", "الاختلاجات،", "التقيو،", "الخمول،", "ورائحة البول الشاذة،", "والعجز عن التغذية،", "البكاء،", "الصراخ،", "التحرك بعنف،", "الدحرجة", "القفز،", "رمي الاشياء،", "يضرب", "يركل", "حبس انفاسهم", "الاعراض الاقتحامية،", "تجنب اي شيء يذكرك بالحدث،", "الشعور بالانفصال،", "غير واقعيين", "الخدر النفسي او الجسدي،", "لا يطيقون انفسهم،", "خائفين", "حساسيتهم تجاه الرفض،", "اكتئاب المزاج،", "سوء التصرف،", "عصبية المزاج", "الشعور بالتفوق", "افتقار التعاطف", "حاجة الي نيل اعجاب", "التقليل من شان انجازات الاخرين،", "المبالغة في تقدير الانجازات الشخصية", "توهم", "استغلال الاخرين،", "حسد الاخرين،", "فقدان الشهية،", "الادمان", "التحرش", "تخريب ممتلكاتهم،", "خرق القانون،", "القاء اللوم", "اختلاق الاعذار", "غير مسوول", "عدواني", "ثغرات في الذاكرة", "هفوات في الذاكرة", "يستفرغ", "اعادة مضغ الطعام المستفرغ", "بصقه", "اعادة بلعه", "يرفضون الدعاية،", "يتجنبون الاجتماعات،", "يتجنبون تكوين صداقات جديدة", "الخوف من ان يتعرضوا للهجر،", "يتحرون عن الشتائم", "صعوبة ابداء الاختلاف", "تحمل الاعتداء الجسدي او العاطفي او الجنسي", "طلب المشورة", "الافتقار الي الثقة", "الخوف من عدم قدرتهم علي الاعتناء بانفسهم", "يقومون بالتدقيق", "لا يسامحون", "يشعرون بالحاجة الي الاكتفاء الذاتي", "السيطرة،", "الحيوية،", "الحماس،", "الجاذبية،", "يبحثون عن جلب الانتباه،", "الضجر،", "يتاثرون بالاخرين", "مختلفون", "تجنب التواصل البصري؛،", "هلاوس", "توهمات", "زور", "اعتقادات خاطئة", "جنون الارتياب،", "ينبض القلب بسرعة،", "الدوخة", "الخدر،", "الجنون،", "لهاث", "نواح", "رهاب الميادين،", "انطوائيين", "وانتحاريين", "الالم", "الضعف،", "الاعراض البدنية", "وصعوبة التنفس", "يتجنبون تناول بعض الاطعمة،", "نقص غذائي", "تناول الشخص المصاب به مواد غير صالحة للاستهلاك", "تناول الورق والطين والاوساخ والشعر،", "الاستحمام بشكل مفرط،", "تنظيف اسنانهم،", "شم انفسهم", "الشحوب،", "توقف التنفس،", "وفقدان الوعي،", "تصوير الاشخاص بدون موافقتهم في اثناء خلعهم لملابسهم", "يرفضون الذهاب الي المدرسة", "اعراض بدنية،", "متلازمة توريت،", "اضطراب الوسواس القهري،", "اضطرابات التعلم", "اضطراب نقص الانتباه وفرط النشاط", "عرض الاعضاء التناسلية امام الغرباء", "ملامسة الاناث للذكور او الاناث الاخرين،", "التعب", "الم البطن", "الاوهام", "الجمود", "الكلام المشوش", "لعق الشفتين", "رعشة في الجفنين", "حركات مضغ", "فرك الاصابع،", "حركات صغيرة باليدين", "قلة التفاعل", "الاحراج", "الايماء بالراس", "الامتناع عن الكلام", "الاشارة باليدين", "الهلوسة،", "المزاج المتقلب،", "الاحساس بفرط النشاط،", "الحساسية الشديدة تجاه النقد،", "اضطرابات في النوم،", "الاعجاب الشديد بالنفس،", "كثرة الكلام", "الثرثرة", "العصبية-", "عدم القدرة علي التاقلم،", "الكذب", "استمرارية الكذب", "الاعتماد المرضي", "فقدان العلاقات", "صعوبة في التعبير", "تجنب المواقف", "ضعف الثقة بالنفس", "تنفيذ رغبات الاخرين لتلقي المديح", "عدم القدرة علي الاهتمام بالنفس", "لارتياب الدائم", "سوء نية", "الخوف من استغلال الاخرين", "تحريف الاحداث", "عدم تقبل النقد", "غياب التسامح", "القسوة", "الحسد", "الغيرة", "الشك المرضي", "تفسير المواقف بشكل سلبي،", "عدائي،", "الغرور،", "التكبر،", "يجلس فوق السرير", "يفتح عينيه", "عيناه محدقتين", "لا يستجيب،", "صعب ايقاظه", "مشتتا لمدة قصيرة", "لا يتذكر نوبة السير اثناء النوم،", "مشكلات في اداء", "نوبات هلع", "الصراخ", "تحريك الذراعين والساقين", "قيادة سيارة،", "مغادرة المنزل", "سلوك غير معتاد", "نشاط جنسي دون وعي", "التعرض للاصابة", "التعامل بعنف", "فترة الارتباك", "السير اثناء النوم", "يتخلص الجسم من العقار", "لبسة الجنس الاخر", "غير التحفيز الجنسي", "للحد من القلق", "الاسترخاء", "تجربة الجانب الانثوي", "توقع الشر", "شعور دائم بعدم الكفاية", "القلق من الفشل", "التفكير السلبي المستمر،", "انتقاد الذات", "التردد،", "صعوبة في الترقية،", "تجنب التحديات،", "خوفا من الفشل", "والاحتياجات", "تدهور العلاقات الاجتماعية", "التردد", "الامتنان تجاه الشخص المعتدي،", "مشاعر سلبية لدي الضحية نحو اي شخص يحاول مساعدته او انقاذه", "التعاطف", "الدعم والمساندة للمعتدي", "تاييد تصرفات وافكار المعتدي", "العدوانية", "اجابات غير منطقية", "خاطئة عمدا", "ادعاء الاصابة بامراض،", "الام غير حقيقية", "الرغبة في جذب انتباه الاخرين", "واثارة تعاطفهم", "الانسحاب الاجتماعي", "تفضيل ارتداء ملابس الجنس الاخر", "الاصرار علي انهم من الجنس الاخر", "ينتمون للجنس الاخر", "المشاركة في الالعاب والانشطة المرتبطة بالجنس الاخر", "مشاعر سلبية تجاه اعضائهم التناسلية", "القيام بعمل يرتبط عادة بهذا الجنس او الزواج من شخص بالجنس", "انكار مشاعرهم", "انكار مشاعرهم بالرغبة لان يكونوا من الجنس الاخر", "يقرون بهويتهم من الجنس الاخر", "خوف", "قلق ملحوظ", "التجنب", "الغمغمة بوتيرة اقل", "اصدار اصوات بدرجة اقل من المعتاد", "نطق بالكلمات الاولي متاخرا", "اختلالات الحروف الساكنة والمتحركة،", "كثرة اغفال اصوات عند التحدث", "اخطاء صوتية", "صعوبة الانتقال بسلاسة من احد الاصوات او المقاطع الصوتية او الكلمات", "اختلالات الحروف المتحركة،", "فصل المقاطع", "عدم الاتساق", "كلام يصعب فهمه", "الوقفات الموقتة بين المقاطع الصوتية", "اضطراب حركة الفك او الشفتين او اللسان", "تشديد الحروف", "الندوب", "الجروح القطعية الحديثة", "لخدوش", "الكدمات", "علامات العض", "الجروح", "فرك منطقة ما من الجلد بشكل مفرط", "احداث تسلخ", "ابقاء الادوات الحادة او الادوات المستخدمة لايذاء النفس", "مشاكل في العلاقات", "لسلوكيات والانفعالات المندفعة والحادة وغير المتوقعة والمتغيرة", "قلة الحيلة", "فقدان الامل", "الاحساس بالضالة", "صعوبة في فهم الكلام", "صعوبة في القراءة او الكتابة", "مشكلات في التحدث", "صعوبة في تكرار الكلمات", "صعوبة بدء النطق بكلمة او عبارة او جملة", "اطالة كلمة", "اصوات داخل الكلمة", "تكرار صوت او مقطع او كلمة", "الصمت برهة لنطق مقاطع او كلمات معينة", "التوقف موقتا", "اضافة اصوات حشو", "التوتر", "التشنج", "عدم القدرة علي التواصل", "القلق حيال التكلم", "رمش العينين بسرعة", "ارتعاش الشفتين او الفك", "حركات غير طبيعية في الوجه", "الايماء بالراس الي اعلي واسفل", "احكام القبضتين", "الاكتئاب", "الشعور بالهياج", "افكار انتحارية", "العدوانية", "سهولة الغضب", "الخرف", "مشكلة في التخطيط او التنظيم", "اخرقا", "صعوبة في التحدث بوضوح", "الانفصال عن ذاتك وعواطفك", "اشخاص واشياء غير حقيقي", "الاحساس غير الواضح بهوية الشخصية", "التوتر الشديد", "عدم القدرة علي التاقلم", "التوتر العاطفي او المهني", "فقدان الذاكرة", "نسيان", "الاكتئاب", "السلوكيات الانتحارية", "الغضب", "سهولة الاستثارة", "زيادة التوتر والطاقة", "تسارع الافكار", "الوخز", "الارتعاش", "تسارع ضربات القلب او خفقانه بشدة", "ضيق الصدر", "صعوبة في النوم ليلا", "الاستيقاظ خلال الليل", "الاستيقاظ مبكرا جدا", "الشعور بالتعب", "النعاس اثناء النهار", "المزاج السيئ", "صعوبة في الانتباه او التركيز", "ارتكاب اخطاء اكثر", "قلق مستمر بشان النوم", "القلق", "تنام في اوقات ليست طبيعية", "الشعور بالنعاس الشديد", "مواجهة صعوبة في النوم", "الاستيقاظ في وقت مبكر جدا", "عدم القدرة علي معاودة النوم", "الاستيقاظ اثناء الليل", "الشخير", "الاختناق", "توقف التنفس موقتا", "تشعر بوخز خفيف", "فعل حركات تزعجك اثناء النوم", "التنفس بطريقة غير طبيعية", "الشعور برغبة مزعجة في التحرك", "المشي اثناء النوم", "الانشطة غير المعتادة اثناء النوم", "مشكلة في الانتباه", "مشاكل في الكلام او اللغة", "مشكلة في فهم الاتجاهات", "مشكلة في تذكر الاشياء", "مشكلة في استخدام اليد والاصبع", "الاحباط مشاكل سلوكية", "ببطء في البدء بالتحدث وتسمية الحروف والصور", "مشكلة في اخراج اصوات الكلمات", "مشكلة في روية الكلمات الفردية", "البطء في القراءة بصوت عال", "اخطاء املائية وكتابية", "فقدان الشهية", "النهام الشهية المفرطة", "اضطراب نهم الطعام", "فقدان الامل", "عدم الاستمتاع", "الشعور بالحزن", "البكاء المتكرر", "لقلق", "التوتر", "العصبية", "الضغط", "الانفعال", "صعوبة في النوم", "عدم تناول ما يكفي من الطعام", "صعوبة في التركيز", "صعوبة في اداء الانشطة اليومية", "الابتعاد عن العائلة والاصدقاء", "تجنب اشياء مهمة", "التفكير في الانتحار", "التصرف بمشاعر انتحارية", "الغم الشديد", "عدم القدرة علي القيام بمهامه", "تتدهور علاقاته الشخصية واداوه الوظيفي", "الخوف من المرض", "لفحص انفسهم مرارا وتكرارا", "يتجنبون النشاطات", "اضطراب جسدي", "مشاعر الخوف", "ظهور اية احاسيس جديدة", "متنبهين بدرجة كبيرة للاصابة بالمرض", "يطلبون الطمانة باستمرار", "القلق اللامتناهي", "توتر العلاقات الاجتماعية", "الاعتداء الجنسي علي الاطفال", "لمس الاعضاء التناسلية", "القيام بالجماع", "ينجذبون الي الاولاد الصغار", "يخشون التعرض الي السخرية", "يشعرون بالارتباك", "حمرار الوجه", "التعرق", "يعجزون عن الحركة", "ينكفئون علي انفسهم", "يتشبثون بابائهم", "يرفضون الذهاب الي المدرسة او حضور المناسبات الاجتماعية", "الم في المعدة", "صداع", "الهم والخوف من الاحراج", "تجاهل الاخرين", "الثقة العمياء بالنفس", "الغطرسة", "التشبث بالراي", "قد يكونون ساحرين ومقنعين في جهودهم للحصول علي ما يريدون", "السلوك الاندفاعي الاندفاعية", "عدم المسوولية", "عدم ابداء اي استجابة عند منحها", "الخوف", "الحزن", "سهولة الاستثارة", "الانعزال", "مظهر حزين", "وخامل", "عدم طلب الراحة", "العجز عن الابتسام", "متابعة الاخرين باهتمام دون المشاركة في التفاعل الاجتماعي معهم", "العجز عن التواصل مع غيره", "عدم الرغبة في لعب", "المشكلات السلوكية", "العجز عن طلب الدعم او المساعدة", "التهام الطعام بسرعة", "الشعور بالامتلاء المزعج", "تناول كميات كبيرة من الطعام حتي عند عدم وجود شعور بالجوع", "تناول الطعام بمفردهم", "يشعرون بالحرج من الاخرين", "الشعور بالاشمئزاز", "الاكتئاب", "الذنب بعد الافراط في تناول الطعام", "بتناول الملينات او مدرات البول الادوية", "الصيام", "القاء", "الافراغ", "التقيو الذاتي", "نظام غذائي صارم", "الافراط في ممارسة التمارين", "الشكوي الدائمة من السمنة", "انكار نحول الجسد وهزاله", "التفكير في الطعام بشكل دائم", "قياس كميات الطعام المتناولة", "تخزين", "اخفاء", "رمي الطعام", "تخطي وجبات الطعام", "التظاهر بتناول الطعام", "اللجوء للكذب حول مقدار الطعام المتناول", "ممارسة الرياضة بشكل قهري", "ارتداء الملابس الضخمة", "وزن الجسم عدة مرات في اليوم", "الاعتماد في احترام النفس علي مدي النحول", "للكهارل الكهربائية مثل الصوديوم والبوتاسيوم والكلوريد", "يضعف القلب", "نظم القلب غير طبيعي", "الجفاف", "عرضة للاغماء", "تنخفض مستويات البوتاسيوم والصوديوم في الدم", "الشعور بعدم الاهمية", "الذنب", "الوحدة", "بالياس", "تغيرات في عادات النوم", "صعوبة النوم", "الاستيقاظ في الصباح الباكر", "نقص او زيادة الوزن", "اضطراب القدرة علي التركيز", "زيادة معدل استهلاك الكحول", "التدخين", "المخدرات", "التفكير بالموت والانتحار،", "تراجع الاهتمام", "عدم الشعور بالسعادة", "رغبة قوية بالاستحواذ", "يشعر بالضائقة الشديدة", "الفوضي", "المنزل محتشرا بالافات والحشرات الضارة", "اكتناز الحيوانات", "القلق من مظهرهم", "الوسواس", "شعور الارتياح", "ضيق", "فقدان السيطرة", "النزف", "سحب الشعر", "يلفوا الشعر بين اصابعهم", "يسحبوا الخصل بين اسنانهم", "يعضون الشعر", "ابتلاع شعرهم", "الغثيان", "القيء", "الالم", "نتف الشعر", "يقضمون اظافرهم", "يمضغون خدودهم", "الحرج", "الخجل", "عجزهم عن السيطرة علي سلوكهم", "فقدان الشعر", "يشدون الشعر", "يشدون الخيوط من الملابس او البطانيات او من المنسوجات", "القروح", "المناطق المتندبة", "الندوب", "القروح", "يخدشون جلدهم تلقائيا", "الانشطة الطقوس خدش الجلد", "مضغ خدهم", "يخدش بعض الاشخاص جلد الاخرين", "ضرر في الجلد", "تمويه الضرر الجلدي", "حالات عدوي", "نزفا شديدا", "اضطراب الوسواس القهري", "اضطراب شد الشعر", "الاكتئاب", "هواجس او افكار او صور او رغبات ملحة تراودهم مرارا وتكرارا", "الحاق الضرر بالنفس او الاخرين", "التنظيف", "التلوث", "الافكار الممنوعة او المحرمة", "الهواجس العدوانية او الجنسية", "التناظر", "المخاوف", "القلق من ان لمس المقابض", "الشكوك", "الشعور بالاكراه", "لمنع او تخفيف القلق الناجم عن هواجسهم", "الغسل", "الترتيب", "الفحص المتكرر", "غسل اليدين المفرط", "العد المتكرر في الذهن", "التمتمة الهادئة", "لوسواس الاستحمام", "تجنب للقذراة", "يستحم لساعات كل يوم", "تتسبب في الكثير من الضيق", "تستغرق كل الوساوس والطقوس وقتا طويلا", "افكارهم الوسواسية", "سلوكهم القهري مفرط", "تتدهور علاقاتهم", "لا يكون اداوهم جيدا في المدرسة", "هواجسهم ومشاعرهم القهرية غير عقلانية", "غير عقلانية"


    ],
    "treatment": [
        "العلاج النفسي", "مضادات الاكتئاب", "الاسترخاء،", "الليثيوم", "الادوية", "فالبروات،", "التخليج الكهربي", "العلاج السلوكي", "العلاج الفردي واحيانا الجماعي", "المعالجة النفسية", "العلاج الجماعي،", "طرائق تدبير السلوك،", "العلاج النفسي،", "الادخال الي المستشفي", "العلاج النفسي الفردي والعائلي", "دعم تعليمي", "العلاج السلوكي", "التواصل", "التدريب المباشر", "العلاج الدوائي", "العلاج اللغوي", "تطوير برنامج فردي شامل للطفل،", "تشتيت انتباه الطفل،", "ابعاد الطفل عن المكان،", "مثبطات استرداد السيروتونين الانتقائية،", "العلاج بالتعرض", "المعالجة النفسية،", "العناية الذاتية،", "العلاج السلوكي المعرفي،", "التنويم", "التخيل الموجه", "الادوية المضادة للقلق والاكتئاب", "طرائق تعديل السلوك،", "الادوية المضادة للذهان", "البنزوديازيبينات", "مثبطات استرداد السيروتونين الانتقائية", "التعديل السلوكي", "برنامج اعادة التاهيل", "تدخلا جراحي", "المعالجة السلوكية المعرفية", "مكملات الحديد", "مجموعات الدعم،", "التداخل السلوكي الشامل لمعالجة العرات", "العلاج النفسي الفردي او الجماعي،", "ريسبيريدون،", "اولانزابين،", "اريبيبرازول،", "العلاج الاسري", "الادوية المضادة للصرع", "ايثوسكسيميد", "فالبروات الصوديوم،", "الحمية الكيتونية", "العلاج النفسي استخدام اللعب والرسم،", "الانسولين المعدل", "الصدمات الكهربائية،", "المهدئات", "العلاج النفسي", "العلاج الدوائي", "تقديم الدعم", "التربية المتوازنة", "الدعم للطفل", "منحه الحرية لتطوير استقلاليته", "جلسات نفسية فردية", "الثقة بالنفس", "الاستقلالية", "زيادة وعيه", "التحلي بالصبر", "الايقاظ الاستباقي", "مضادات الاكتئاب", "التنويم المغناطيسي الذاتي", "التوجيه المعنوي", "وسط هادئ", "البنزوديازيبين", "المضادة للذهان", "مجموعات الدعم", "المجموعات الاجتماعية", "التدرب علي مواجهة المخاوف", "التفكر اكثر في خلق الكون،", "تعلم حب الذات،", "التركيز من السلبيات الي الايجابيات", "بناء علاقات حقيقية", "الابتعاد عن العالم الافتراضي", "الثقة بالنفس", "التعبير الحر عن الذات", "المعالجات السلوكية", "العلاج الهرموني الموكد للجنس", "الجراحة الموكدة للجنس", "التحليل الكهربائي", "المعالجة بالتعرض", "تقنيات الاسترخاء", "التنويم المغناطيسي", "البنزوديازيبين", "حاصرات بيتا", "جلسات متخصصة لتحسين النطق والتواصل،", "علاج التخاطب،", "تمارين النطق المنزلية،", "تمارين يومية لتعزيز مهارات النطق،", "طرق التواصل البديلة،", "نمط الحياة والعلاجات المنزلية،", "التاقلم والدعم،", "جلسات علاج النطق", "اجهزة التواصل المساعدة", "معالجة النطق", "اجهزة الكترونية", "العلاج السلوكي المعرفي", "التفاعل بين الوالدين والطفل", "ممارسة نشاطات روتينية ومنتظمة", "العيش في بيئة امنة ومنظمة", "المشورة", "المعالجة بالمحادثة", "الادوية", "مضادات الاكتئاب", "مثبطات اعادة امتصاص السيروتونين الانتقائية", "الاقراص المنومة", "المعالجة بالضغط الموجب المستمر", "خيارات نمط الحياة الصحي", "المكملات الغذائية", "برامج تعلم", "الكتب الصوتية", "التغذية السليمة", "اتباع عادات غذائية صحية", "العلاج بالحوار", "العلاج السلوكي", "العلاج النفسي الفردي او الجماعي", "المعالجة بالتعرض", "مثبطات استرداد السيروتونين الانتقائية", "نظام المكافات", "المعالجة النفسية", "الادوية المنبهة", "ادوية انقاص الوزن", "مثبطات الشهية", "برامج انقاص الوزن السلوكية", "العلاج الاسري", "الفحوصات المنتظمة", "التنويم", "المكملات السائلة", "علاج الاضطرابات الصحية النفسية", "اولانزابين", "تناول الطعام الصلب", "الاستشارات الغذائية", "العلاج النفسي الشخصي والعائلي", "المشي", "زيارات اجتماعية", "علاج كسر العادة", "العلاج المعرفي", "الوقاية من ردة الفعل", "العلاج بالتعرض"
    ]
}

def annotate_named_entities(text, named_entities):
    annotated_text = text
    for category, entities in named_entities.items():
        for entity in entities:
            if entity in annotated_text:
                annotated_text = annotated_text.replace(entity, f"[{entity}]({category.upper()})")
    return annotated_text

with open("segmented_data.json", "r", encoding="utf-8") as f:
    segmented_data = json.load(f)

annotated_data = []
for entry in segmented_data:
    annotated_entry = {
        "disorder": [annotate_named_entities(sentence, {"disorder": named_entities["disorder"]}) for sentence in entry.get("disorder", [])],
        "symptom": [annotate_named_entities(sentence, {"symptom": named_entities["symptom"]}) for sentence in entry.get("symptom", [])],
        "treatment": [annotate_named_entities(sentence, {"treatment": named_entities["treatment"]}) for sentence in entry.get("treatment", [])]
    }
    annotated_data.append(annotated_entry)

with open("NamedEntities.json", "w", encoding="utf-8") as f:
    json.dump(annotated_data, f, ensure_ascii=False, indent=4)
print("file complete! Results saved to 'NamedEntities.json'.")

file complete! Results saved to 'NamedEntities.json'.


In [38]:
!pip install nltk

In [19]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [39]:
import json
import nltk
import re
from nltk.tokenize import word_tokenize

nltk.download('punkt')

with open("segmented_data.json", "r", encoding="utf-8") as f:
    segmented_data = json.load(f)

def tokenize_text(text):
    words = word_tokenize(text)
    refined_words = []

    for word in words:
        match = re.match(r'^(و)?(\w+)$', word)
        if match:
            prefix, core_word = match.groups()
            if prefix:
                refined_words.append(prefix)
            refined_words.append(core_word)
        else:
            refined_words.append(word)

    return refined_words

tokenized_data = []
for entry in segmented_data:
    tokenized_entry = {
        "disorder": [tokenize_text(sentence) for sentence in entry["disorder"]],
        "symptom": [tokenize_text(sentence) for sentence in entry["symptom"]],
        "treatment": [tokenize_text(sentence) for sentence in entry["treatment"]]
    }
    tokenized_data.append(tokenized_entry)

with open("tokenized_data.json", "w", encoding="utf-8") as f:
    json.dump(tokenized_data, f, ensure_ascii=False, indent=4)

print("Tokenization complete! Results saved to 'tokenized_data.json'.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenization complete! Results saved to 'tokenized_data.json'.


*Manual Annotation*

In [41]:
import json
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

ENTITY_TYPES = ["DISORDER", "SYMPTOM", "TREATMENT", "O"]

arabic_text = "تعاني الطفلة من اضطراب القلق الاجتماعي، وتواجه صعوبة في التحدث مع الآخرين، مما يؤثر على أدائها الدراسي. أوصى الأخصائي بالعلاج النفسي السلوكي مع مراقبة مستمرة."


# Tokenize the text using NLTK
tokens = word_tokenize(arabic_text)

annotations = []
print("\nAnnotate the following tokens (Choose: DISORDER, SYMPTOM, TREATMENT, O):\n")

prev_entity = "O"
for token in tokens:
    tag = input(f"Token: {token} → Enter entity type {ENTITY_TYPES}: ").strip().upper()

    if tag in ENTITY_TYPES and tag != "O":
        iob_tag = "B-" + tag if prev_entity != tag else "I-" + tag
    else:
        iob_tag = "O"

    annotations.append((token, iob_tag))
    prev_entity = tag if tag in ENTITY_TYPES else "O"

df = pd.DataFrame(annotations, columns=["Token", "Tag"])
df.to_csv("arabic_psychology_corpus.csv", index=False, encoding="utf-8")


json_data = {"tokens": [t[0] for t in annotations], "tags": [t[1] for t in annotations]}
with open("arabic_psychology_corpus.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

print("\n Annotation complete! Data saved as 'arabic_psychology_corpus.csv' and 'arabic_psychology_corpus.json'.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Annotate the following tokens (Choose: DISORDER, SYMPTOM, TREATMENT, O):

Token: تعاني → Enter entity type ['DISORDER', 'SYMPTOM', 'TREATMENT', 'O']: O
Token: الطفلة → Enter entity type ['DISORDER', 'SYMPTOM', 'TREATMENT', 'O']: O
Token: من → Enter entity type ['DISORDER', 'SYMPTOM', 'TREATMENT', 'O']: O
Token: اضطراب → Enter entity type ['DISORDER', 'SYMPTOM', 'TREATMENT', 'O']: DISORDER
Token: القلق → Enter entity type ['DISORDER', 'SYMPTOM', 'TREATMENT', 'O']: DISORDER
Token: الاجتماعي، → Enter entity type ['DISORDER', 'SYMPTOM', 'TREATMENT', 'O']: DISORDER
Token: وتواجه → Enter entity type ['DISORDER', 'SYMPTOM', 'TREATMENT', 'O']: O
Token: صعوبة → Enter entity type ['DISORDER', 'SYMPTOM', 'TREATMENT', 'O']: SYMPTOM
Token: في → Enter entity type ['DISORDER', 'SYMPTOM', 'TREATMENT', 'O']: O
Token: التحدث → Enter entity type ['DISORDER', 'SYMPTOM', 'TREATMENT', 'O']: SYMPTOM
Token: مع → Enter entity type ['DISORDER', 'SYMPTOM', 'TREATMENT', 'O']: O
Token: الآخرين، → Enter entity typ